# Simple

A replica of the C++ version to be found in ../src/roq/samples/simple/

In [1]:
import logging

In [2]:
# Do NOT enable logging unless you REALLY need it! And then DO NOT SAVE this workbook!
# logging.basicConfig(level=logging.INFO)

## Import the Roq Python wrapper

### Note!

You will have to follow these steps before launching Jupyter

* Install Miniconda from https://docs.conda.io/en/latest/miniconda.html
    * `wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh`
    * `bash Miniconda3-latest-Linux-x86_64.sh -b`
    * `source ~/miniconda3/bin/activate'
* Install the Roq Python package from https://roq-trading.com/conda/unstable
    * `conda install --override-channels --channel defaults --channel https://roq-trading.com/conda/unstable roq-python`

In [3]:
import roq

## The event handler interface: roq.client.Handler

In [4]:
help(roq.client.Handler)

Help on class Handler in module roq.client:

class Handler(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      Handler
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: roq.client.Handler) -> None
 |  
 |  cancel_order(...)
 |      cancel_order(self: roq.client.Handler, account: str, order_id: int, source: int) -> None
 |  
 |  create_order(...)
 |      create_order(self: roq.client.Handler, account: str, order_id: int, exchange: str, symbol: str, side: roq.Side, quantity: float, order_type: roq.OrderType, limit_price: float, time_in_force: roq.TimeInForce, position_effect: roq.PositionEffect, order_template: str, source: int) -> None
 |  
 |  modify_order(...)
 |      modify_order(self: roq.client.Handler, account: str, order_id: int, quantity_change: float, limit_price: float, source: int) -> None
 |  
 |  on_cancel_order_ack(...)
 |      on_cancel_order_ack(self: roq.clie

## A trivial implementation of the event handler interface

Enable logging (see above) if you want to see each event.

In [5]:
class Strategy(roq.client.Handler):
    """
    A simple strategy implementing the client interface.
    """

    def __init__(
            self,
            account: str,
            exchange: str,
            symbol: str):
        """
        Constructor
        """
        roq.client.Handler.__init__(self)  # important! see pybind11 doc
        self.account = account
        self.exchange = exchange
        self.symbol = symbol

    def on_start(self):
        """ Event handler """
        logging.info('on_start()')

    def on_stop(self):
        """ Event handler """
        logging.info('on_stop()')

    def on_connection_status(
            self,
            source: int,
            source_name: str,
            connection_status: roq.ConnectionStatus):
        """ Event handler """
        logging.info(
            'on_connection_status(' \
                    'source=%d source_name=%s connection_status=%s)',
            source,
            source_name,
            connection_status)
        if connection_status == roq.ConnectionStatus.CONNECTED:
            self.subscribe(
                accounts={
                    'A1'
                },
                symbols_by_exchange={
                    self.exchange: {
                        self.symbol
                    }
                },
                source=source)

    def on_batch_begin(
            self,
            message_info: roq.MessageInfo):
        """ Event handler """
        logging.info(
            'on_batch_begin(message_info=%s)',
            message_info)

    def on_batch_end(
            self,
            message_info: roq.MessageInfo):
        """ Event handler """
        logging.info(
            'on_batch_end(message_info=%s)',
            message_info)

    def on_market_data_status(
            self,
            message_info: roq.MessageInfo,
            market_data_status: roq.MarketDataStatus):
        """ Event handler """
        logging.info(
            'on_market_data_status(' \
                    'message_info=%s market_data_status=%s)',
            message_info,
            market_data_status)

    def on_order_manager_status(
            self,
            message_info: roq.MessageInfo,
            order_manager_status: roq.OrderManagerStatus):
        """ Event handler """
        logging.info(
            'on_order_manager_status(' \
                    'message_info=%s order_manager_status=%s)',
            message_info,
            order_manager_status)

    def on_download_begin(
            self,
            message_info: roq.MessageInfo,
            download_begin: roq.DownloadBegin):
        """ Event handler """
        logging.info(
            'on_download_begin(message_info=%s download_begin=%s)',
            message_info,
            download_begin)

    def on_download_end(
            self,
            message_info: roq.MessageInfo,
            download_end: roq.DownloadEnd):
        """ Event handler """
        logging.info(
            'on_download_end(message_info=%s download_end=%s)',
            message_info,
            download_end)

    def on_reference_data(
            self,
            message_info: roq.MessageInfo,
            reference_data: roq.ReferenceData):
        """ Event handler """
        logging.info(
            'on_reference_data(message_info=%s reference_data=%s)',
            message_info,
            reference_data)

    def on_market_status(
            self,
            message_info: roq.MessageInfo,
            market_status: roq.MarketStatus):
        """ Event handler """
        logging.info(
            'on_market_status(message_info=%s market_status=%s)',
            message_info,
            market_status)

    def on_session_statistics(
            self,
            message_info: roq.MessageInfo,
            session_statistics: roq.SessionStatistics):
        """ Event handler """
        logging.info(
            'on_session_statistics(' \
                    'message_info=%s session_statistics=%s)',
            message_info,
            session_statistics)

    def on_daily_statistics(
            self,
            message_info: roq.MessageInfo,
            daily_statistics: roq.DailyStatistics):
        """ Event handler """
        logging.info(
            'on_daily_statistics(message_info=%s daily_statistics=%s)',
            message_info,
            daily_statistics)

    # on_market_by_price

    def on_trade_summary(
            self,
            message_info: roq.MessageInfo,
            trade_summary: roq.TradeSummary):
        """ Event handler """
        logging.info(
            'on_trade_summary(message_info=%s trade_summary=%s)',
            message_info,
            trade_summary)

    def on_position_update(
            self,
            message_info: roq.MessageInfo,
            position_update: roq.PositionUpdate):
        """ Event handler """
        logging.info(
            'on_position_update(message_info=%s position_update=%s)',
            message_info,
            position_update)

    def on_order_update(
            self,
            message_info: roq.MessageInfo,
            order_update: roq.OrderUpdate):
        """ Event handler """
        logging.info(
            'on_order_update(message_info=%s order_update=%s)',
            message_info,
            order_update)

    def on_trade_update(
            self,
            message_info: roq.MessageInfo,
            trade_update: roq.TradeUpdate):
        """ Event handler """
        logging.info(
            'on_trade_update(message_info=%s trade_update=%s)',
            message_info,
            trade_update)

    def on_create_order_ack(
            self,
            message_info: roq.MessageInfo,
            create_order_ack: roq.CreateOrderAck):
        """ Event handler """
        logging.info(
            'on_create_order_ack(message_info=%s create_order_ack=%s)',
            message_info,
            create_order_ack)

    def on_modify_order_ack(
            self,
            message_info: roq.MessageInfo,
            modify_order_ack: roq.ModifyOrderAck):
        """ Event handler """
        logging.info(
            'on_modify_order_ack(message_info=%s modify_order_ack=%s)',
            message_info,
            modify_order_ack)

    def on_cancel_order_ack(
            self,
            message_info: roq.MessageInfo,
            cancel_order_ack: roq.CancelOrderAck):
        """ Event handler """
        logging.info(
            'on_cancel_order_ack(message_info=%s cancel_order_ack=%s)',
            message_info,
            cancel_order_ack)

## A simple wrapper function so we can time it

In [6]:
def run_simulation(connections, account, exchange, symbol):
    """
    Simulation example.
    """
    # creates the strategy object
    strategy = Strategy(
        account=account,
        exchange=exchange,
        symbol=symbol)
    # creates an event generator object
    generator = roq.client.Simulation.create_generator(
        'csv-old',
        connections[0])  # FIXME(thraneh): support vector
    # creates an order matching object
    matcher = roq.client.Simulation.create_matcher(
        'simple',
        'CFFEX')
    # runs the simulation
    roq.client.Simulation(strategy, generator, matcher).dispatch()

In [7]:
## Variables

In [8]:
HISTORY = 'src/roq/samples/simple/history_2018-02-23.csv'  # about 372k market data events (a full trading day)

In [9]:
ACCOUNT = 'A1'  # not really important unless you switch to trading
EXCHANGE = 'CFFEX'  # China Financial Futures EXchange
SYMBOL = 'IC1803'  # the most liquid instrument (on the specific trading day)

## Run the simulation and time it

In [10]:
%timeit run_simulation(connections=[HISTORY,], account=ACCOUNT, exchange=EXCHANGE, symbol=SYMBOL)

RuntimeError: No such file or directory